In [ ]:
input_file_path = "/Users/ydubey/Desktop/transcription/241118_1543.mp3"

In [1]:
import ffmpeg

input_file = input_file_path
output_file = input_file_path.replace(".mp3", ".wav")

try:
    stream = ffmpeg.input(input_file)
    stream = ffmpeg.output(stream, output_file)
    ffmpeg.run(stream)
    print("Conversion successful")
except ffmpeg.Error as e:
    print(f"Error: {e.stderr.decode()}")

Conversion successful


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/opt/anaconda3/envs/transcription --cc=arm64-apple-darwin20.0.0-clang --ar=arm64-apple-darwin20.0.0-ar --nm=arm64-apple-darwin20.0.0-nm --ranlib=arm64-apple-darwin20.0.0-ranlib --strip=arm64-apple-darwin20.0.0-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libdav1d --enable-zlib --enable-libaom --enable-pic --enable-shared --disable-static --disable-gpl --enable-version3 --disable-sdl2 --enable-libopenh264 --enable-libopus --enable-libmp3lame --enable-libopenjpeg --enable-libvorbis --enable-pthreads --enable-libtesseract --enable-libvpx --enable-librsvg
  libavutil      58. 29.100 / 58. 29.100
  libavcodec     60. 31.102 / 60. 31.102
  libavformat    60. 16.100 

In [12]:
import pandas as pd
import torch
from pyannote.audio import Pipeline
import whisper

# Initialize Pyannote pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="YOUR HUGGING FACE TOKEN")

# Load audio file
audio_file = output_file

# Check if MPS is available and set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

# Send pipeline to the appropriate device
pipeline.to(device)

# Perform diarization
diarization = pipeline(audio_file)

# Load Whisper model and transcribe audio
model = whisper.load_model("base.en")
result = model.transcribe(audio_file)

# Process diarization and transcription results
transcript = {}
assigned_sections = set()  # To track assigned sections

def calculate_overlap(start1, end1, start2, end2):
    """Calculate the overlap between two time intervals."""
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    return max(0, overlap_end - overlap_start)

# Store overlaps for each section
for segment, _, speaker in diarization.itertracks(yield_label=True):
    segment_duration = segment.end - segment.start
    if speaker not in transcript:
        transcript[speaker] = []

    for section in result["segments"]:
        overlap = calculate_overlap(segment.start, segment.end, section["start"], section["end"])
        overlap_fraction = overlap / (section["end"] - section["start"])

        # Store overlap details
        section.setdefault("overlaps", [])
        section["overlaps"].append((speaker, overlap, overlap_fraction))

# Assign text based on overlap conditions
for section in result["segments"]:
    if "overlaps" in section and section["id"] not in assigned_sections:
        overlaps = sorted(section["overlaps"], key=lambda x: x[1], reverse=True)  # Sort by overlap amount
        max_overlap = overlaps[0]

        if max_overlap[2] > 0.5:
            # Assign to the segment with >50% overlap
            speaker = max_overlap[0]
            transcript[speaker].append((section["start"], section["end"], section["text"]))
            assigned_sections.add(section["id"])
        elif len(overlaps) > 1:
            # Assign to the segment with the most overlap if all are <50%
            speaker = max_overlap[0]
            transcript[speaker].append((section["start"], section["end"], section["text"]))
            assigned_sections.add(section["id"])

# Create DataFrame
df = pd.DataFrame([(speaker, start, end, text) 
                   for speaker, segments in transcript.items() 
                   for start, end, text in segments],
                  columns=["Speaker", "Start", "End", "Text"])

# Sort by start time and save to CSV
df.sort_values(by="Start").reset_index(drop=True).to_csv("transcription_results_modified_3.csv", index=False)

print("Transcription results saved to 'transcription_results.csv'")


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1. Bad things might happen unless you revert torch to 1.x.


INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/opt/anaconda3/envs/transcription/lib/python3.10/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will exec

Using mps


INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/opt/anaconda3/envs/transcription/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary

Transcription results saved to 'transcription_results.csv'
